<a href="https://colab.research.google.com/github/jcho49614/projects/blob/main/modeltrain6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim #optimizer
import torch.nn.functional as F
from torchvision import datasets, transforms #datasets, transforms
from torch.utils.data import DataLoader as DL #DataLoader wrapping it around.
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
!pip install nvidia-ml-py3

In [ ]:
from pynvml import *

def show_cuda_usage():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)  # GPU 0

    mem_info = nvmlDeviceGetMemoryInfo(handle)
    util_info = nvmlDeviceGetUtilizationRates(handle)

    print(f"GPU Memory Used: {mem_info.used / 1024**2:.2f} MB")
    print(f"GPU Memory Total: {mem_info.total / 1024**2:.2f} MB")
    print(f"GPU Utilization: {util_info.gpu}%")
    print(f"Memory Utilization: {util_info.memory}%")

    nvmlShutdown()

In [ ]:
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
  print("Device Name:", torch.cuda.get_device_name(0))
  print("CUDA Version:", torch.version.cuda)
  print("Torch Version:", torch.__version__)

CUDA Available: True
Device Name: Tesla T4
CUDA Version: 12.4
Torch Version: 2.6.0+cu124


In [ ]:
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # mean and std tuples
])

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


In [ ]:
#using CIFAR100!
trainset = torchvision.datasets.CIFAR100(
    root = './data',
    train = True,
    download = True,
    transform=transform_train
)
testset = torchvision.datasets.CIFAR100(
    root = './data',
    train = False,
    download = True,
    transform=transform
)

In [ ]:
trainloader = DL(trainset, batch_size = 15000, shuffle=True, num_workers=20, pin_memory=True)
testloader = DL(testset, batch_size = 15000, shuffle= False, num_workers = 20, pin_memory=True)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# class CNNMODEL(nn.Module):
#   def __init__(self): #self refers to the instance itself!
#     super(CNNMODEL, self).__init__()
#     self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
#     self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#     self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
#     self.pool = nn.MaxPool2d(2,2)
#     self.fc1 = nn.Linear(128*4*4, 128) #flatten to 128
#     self.fc2 = nn.Linear(128,100) #classify to 100 classes.

#   def forward(self, x): #x is just a newly created
#     x = self.pool(F.relu(self.conv1(x)))
#     x = self.pool(F.relu(self.conv2(x)))
#     x = self.pool(F.relu(self.conv3(x)))
#     x = x.view(x.size(0), -1)
#     x = F.relu(self.fc1(x))
#     x = self.fc2(x)
#     return x
#this is one way of writing a cnn module. compact but quite hard to understand.

In [ ]:
class CNNMODEL(nn.Module):
  def __init__(self):
    super(CNNMODEL, self).__init__() #establish the initialization of nn.Module
    self.convolution1 = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=3, padding=1), #padding keeps the size the same -- floor(kernel_size / 2) is best for padding
        nn.ReLU(), #non-linearity
        nn.MaxPool2d(2,2) #divide length, width by half
    )

    self.convolution2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

    self.convolution3 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

    self.convolution4 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size = 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

    self.convolution5 = nn.Sequential(
        nn.Conv2d(128, 256, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

    self.fullyconnected = nn.Sequential(
        nn.Linear(256, 128),
        nn.ReLU()
    )

    self.fc2 = nn.Sequential(
        nn.Linear(128,100)
    )

    self.dropout = nn.Dropout(p=0.5)

# nn.ReLU() needs to be explicitly stated. F.relu() can simply just be mentioned. nn.ReLU() is good for nn.Sequential stuff, F.relu() is good for simply just mentioning it.
#F.relu() is a function, nn.ReLU() is a module, part of nn.Module.

  def forward(self, x):
    x = self.convolution1(x)
    x = self.convolution2(x)
    x = self.convolution3(x)
    x = self.convolution4(x)
    x = self.convolution5(x)
    x = x.view(x.size(0), -1)
    x = self.fullyconnected(x)
    x = self.dropout(x)
    x = self.fc2(x) #last layer does not have a ReLU because it removes negative values to 0.
    return x


In [ ]:
model = CNNMODEL().to(device) #send it to device. make reference called "model".

In [ ]:
criterion = nn.CrossEntropyLoss() #find criterion
optimizer = optim.Adam(model.parameters(), lr=0.001) #use the model parameters, and the learning rate is 0.00001 so it would converge well.
scheduler = ReduceLROnPlateau(
  optimizer,
  mode='min',
  factor=0.01,
  patience=5,
  verbose=False
)


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
num_epochs = 200
for epoch in range(num_epochs):
  print(f"Epoch {epoch + 1} training....", end='')
  model.train() #turn on training mode
  running_loss = 0.0
  for data, labels in trainloader:
    data = data.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    output = model(data) #this refers to x! "data" goes through conv1, relu, pooling, flattening, etc etc.
    loss=criterion(output,labels) #criterion calculates the loss!
    loss.backward() #feed back to model to adjust to optimizer
    optimizer.step() #go forward, make the change, and see if ti works.

    running_loss += loss.item()

    model.eval()
    val_loss=0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
      for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        val_loss += loss.item() * inputs.size(0)
        tmp, preds = torch.max(outputs, 1)
        correct_val += (preds==labels).sum().item()
        total_val += labels.size(0)

    avg_val_loss = val_loss / total_val
    val_acc = correct_val / total_val

  scheduler.step(avg_val_loss)

  print(f"Epoch {epoch+1}/{num_epochs}")
  print(f"Val   Loss: {avg_val_loss:.4f}, Accuracy: {val_acc:.4f}")

  show_cuda_usage()
  print("Done!")




print(f"Epoch {epoch+1}, loss: {running_loss / len(trainloader)}")


Epoch 1 training....Epoch 1/200
Val   Loss: 4.6009, Accuracy: 0.0191
GPU Memory Used: 9001.88 MB
GPU Memory Total: 15360.00 MB
GPU Utilization: 0%
Memory Utilization: 0%
Done!
Epoch 2 training....Epoch 2/200
Val   Loss: 4.5660, Accuracy: 0.0189
GPU Memory Used: 9001.88 MB
GPU Memory Total: 15360.00 MB
GPU Utilization: 0%
Memory Utilization: 0%
Done!
Epoch 3 training....Epoch 3/200
Val   Loss: 4.5087, Accuracy: 0.0230
GPU Memory Used: 12991.88 MB
GPU Memory Total: 15360.00 MB
GPU Utilization: 0%
Memory Utilization: 0%
Done!
Epoch 4 training....Epoch 4/200
Val   Loss: 4.4706, Accuracy: 0.0241
GPU Memory Used: 12991.88 MB
GPU Memory Total: 15360.00 MB
GPU Utilization: 0%
Memory Utilization: 0%
Done!
Epoch 5 training....Epoch 5/200
Val   Loss: 4.4105, Accuracy: 0.0297
GPU Memory Used: 12991.88 MB
GPU Memory Total: 15360.00 MB
GPU Utilization: 0%
Memory Utilization: 0%
Done!
Epoch 6 training....Epoch 6/200
Val   Loss: 4.3315, Accuracy: 0.0451
GPU Memory Used: 12991.88 MB
GPU Memory Total: 1

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for images, labels in testloader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    temp, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

folder_path = '/content/drive/MyDrive/artificialintelligence/Colab Notebooks/Models'


# Make sure the nested directory exists
save_path = os.path.join(folder_path, 'model.pth')

# Example: Save model's state_dict
torch.save(model.state_dict(), save_path)
